# Setup

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/etl-csvs/products_catalog.csv
/kaggle/input/etl-csvs/transactions_log.csv
/kaggle/input/etl-csvs/customers.csv


In [2]:
print("Started ETL process")

customers = pd.read_csv("/kaggle/input/etl-csvs/customers.csv")
print("Loaded customers.csv")
products_catalog = pd.read_csv("/kaggle/input/etl-csvs/products_catalog.csv")
print("Loaded products_catalog.csv")
transactions_log = pd.read_csv("/kaggle/input/etl-csvs/transactions_log.csv")
print("Loaded transactions_log.csv")

Started ETL process
Loaded customers.csv
Loaded products_catalog.csv
Loaded transactions_log.csv


In [3]:
customers.head()

,CustomerID,Business_Category,Business_Size,Customer_Since
0,B2B-2013,Legal Services,Medium,2023-01-26
1,B2B-2310,Tech Startup,Medium,2022-03-08
2,B2B-2782,Tech Startup,Small,2023-09-01
3,B2B-2215,Consulting,Small,2022-10-27
4,B2B-2436,Education,Small,2022-06-24


In [4]:
products_catalog.head()

,SKU,Rev_GL_Class,Sub_Category,Item_Description,Brand,Unit_Price,Attributes
0,1001,OFFICE SUPPLIES,Paper,"PaperOne Paper - A4, 70",PaperOne,37.32,"{""Size"": ""A4"", ""Weight_gsm"": 70}"
1,1002,OFFICE SUPPLIES,Paper,"PaperOne Paper - A4, 80",PaperOne,36.51,"{""Size"": ""A4"", ""Weight_gsm"": 80}"
2,1003,OFFICE SUPPLIES,Paper,"PaperOne Paper - A3, 70",PaperOne,34.99,"{""Size"": ""A3"", ""Weight_gsm"": 70}"
3,1004,OFFICE SUPPLIES,Paper,"PaperOne Paper - A3, 80",PaperOne,61.56,"{""Size"": ""A3"", ""Weight_gsm"": 80}"
4,1005,OFFICE SUPPLIES,Paper,"Double A Paper - A4, 70",Double A,68.41,"{""Size"": ""A4"", ""Weight_gsm"": 70}"


In [5]:
transactions_log.head()

,TransactionID,CustomerID,Date,SKU,Quantity
0,PO-2024-10000,B2B-2734,2024-01-01,1128,2
1,PO-2024-10000,B2B-2734,2024-01-01,1275,2
2,PO-2024-10001,B2B-2154,2024-01-01,1078,19
3,PO-2024-10002,B2B-2398,2024-01-01,1270,1
4,PO-2024-10002,B2B-2398,2024-01-01,1143,3


In [6]:
print(f"""
* Before Merging *
customers.csv shape: {customers.shape[0]} × {customers.shape[1]}
products_catalog.csv shape: {products_catalog.shape[0]} × {products_catalog.shape[1]}
transactions_log.csv shape: {transactions_log.shape[0]} × {transactions_log.shape[1]}
""")


* Before Merging *
customers.csv shape: 800 × 4
products_catalog.csv shape: 370 × 7
transactions_log.csv shape: 50000 × 5



In [7]:
required_columns = {
    "customers": ["CustomerID"],
    "products": ["SKU"],
    "transactions": ["CustomerID", "SKU", "Timestamp"]
}

def check_required_columns(df, required, df_name):
    missing = [col for col in required if col not in df.columns]
    if missing:
        print(f"⚠️ WARNING: {df_name} is missing columns: {missing}")
    else:
        print(f"✅ {df_name} has all required columns.")

check_required_columns(customers, required_columns["customers"], "customers")
check_required_columns(products_catalog, required_columns["products"], "products_catalog")
check_required_columns(transactions_log, required_columns["transactions"], "transactions_log")


✅ customers has all required columns.
✅ products_catalog has all required columns.
⚠️ WARNING: transactions_log is missing columns: ['Timestamp']


In [8]:
transactions_log.rename(columns={'Date': 'Timestamp'}, inplace=True)
transactions_log.head()

,TransactionID,CustomerID,Timestamp,SKU,Quantity
0,PO-2024-10000,B2B-2734,2024-01-01,1128,2
1,PO-2024-10000,B2B-2734,2024-01-01,1275,2
2,PO-2024-10001,B2B-2154,2024-01-01,1078,19
3,PO-2024-10002,B2B-2398,2024-01-01,1270,1
4,PO-2024-10002,B2B-2398,2024-01-01,1143,3


# Lightweight casting

In [9]:
print(customers.columns, products_catalog.columns, transactions_log.columns, sep="\n")

Index(['CustomerID', 'Business_Category', 'Business_Size', 'Customer_Since'], dtype='object')
Index(['SKU', 'Rev_GL_Class', 'Sub_Category', 'Item_Description', 'Brand',
       'Unit_Price', 'Attributes'],
      dtype='object')
Index(['TransactionID', 'CustomerID', 'Timestamp', 'SKU', 'Quantity'], dtype='object')


In [10]:
customers['CustomerID'] = customers['CustomerID'].astype(str)
customers['Business_Category'] = customers['Business_Category'].astype(str)
customers['Business_Size'] = customers['Business_Size'].astype(str)
customers['Customer_Since'] = pd.to_datetime(customers['Customer_Since'])

products_catalog['SKU'] = products_catalog['SKU'].astype(str)
products_catalog['Rev_GL_Class'] = products_catalog['Rev_GL_Class'].astype(str)
products_catalog['Sub_Category'] = products_catalog['Sub_Category'].astype(str)
products_catalog['Item_Description'] = products_catalog['Item_Description'].astype(str)
products_catalog['Brand'] = products_catalog['Brand'].astype(str)
products_catalog['Unit_Price'] = pd.to_numeric(products_catalog['Unit_Price'], errors='coerce')
products_catalog['Attributes'] = products_catalog['Attributes'].astype(str)

transactions_log['TransactionID'] = transactions_log['TransactionID'].astype(str)
transactions_log['CustomerID'] = transactions_log['CustomerID'].astype(str)
transactions_log['Timestamp'] = pd.to_datetime(transactions_log['Timestamp'])
transactions_log['SKU'] = transactions_log['SKU'].astype(str)
transactions_log['Quantity'] = pd.to_numeric(transactions_log['Quantity'], downcast='integer', errors='coerce')

print("Lightweight type casting completed")

Lightweight type casting completed


# Merging

In [11]:
if 'CustomerID' in transactions_log.columns and 'CustomerID' in customers.columns:
    merged_df = pd.merge(transactions_log, customers, on='CustomerID', how='left')
else:
    print("❌ Cannot merge transactions and customers: 'CustomerID' missing.")

merged_df.head()

,TransactionID,CustomerID,Timestamp,SKU,Quantity,Business_Category,Business_Size,Customer_Since
0,PO-2024-10000,B2B-2734,2024-01-01,1128,2,Consulting,Small,2023-05-16
1,PO-2024-10000,B2B-2734,2024-01-01,1275,2,Consulting,Small,2023-05-16
2,PO-2024-10001,B2B-2154,2024-01-01,1078,19,Construction,Small,2022-11-23
3,PO-2024-10002,B2B-2398,2024-01-01,1270,1,Consulting,Small,2022-11-14
4,PO-2024-10002,B2B-2398,2024-01-01,1143,3,Consulting,Small,2022-11-14


In [12]:
if 'SKU' in merged_df.columns and 'SKU' in products_catalog.columns:
    merged_df = pd.merge(merged_df, products_catalog, on='SKU', how='left')
else:
    print("❌ Cannot merge merged_df and products_catalog: 'SKU' missing.")

merged_df.head()

,TransactionID,CustomerID,Timestamp,SKU,Quantity,Business_Category,Business_Size,Customer_Since,Rev_GL_Class,Sub_Category,Item_Description,Brand,Unit_Price,Attributes
0,PO-2024-10000,B2B-2734,2024-01-01,1128,2,Consulting,Small,2023-05-16,COMPUTER SUPPLIES,Laptops,"Dell Laptops - Ryzen 7, 8, 1024",Dell,674.02,"{""CPU"": ""Ryzen 7"", ""RAM_GB"": 8, ""Storage_GB"": ..."
1,PO-2024-10000,B2B-2734,2024-01-01,1275,2,Consulting,Small,2023-05-16,COMPUTER SUPPLIES,Accessories,"Anker Accessories - Headset, Wireless",Anker,422.74,"{""Type"": ""Headset"", ""Connectivity"": ""Wireless""}"
2,PO-2024-10001,B2B-2154,2024-01-01,1078,19,Construction,Small,2022-11-23,OFFICE SUPPLIES,Filing,"Generic Filing - Plastic, Red",Generic,421.81,"{""Material"": ""Plastic"", ""Color"": ""Red""}"
3,PO-2024-10002,B2B-2398,2024-01-01,1270,1,Consulting,Small,2022-11-14,COMPUTER SUPPLIES,Accessories,"Anker Accessories - Keyboard, Wired",Anker,403.19,"{""Type"": ""Keyboard"", ""Connectivity"": ""Wired""}"
4,PO-2024-10002,B2B-2398,2024-01-01,1143,3,Consulting,Small,2022-11-14,COMPUTER SUPPLIES,Laptops,"Dell Laptops - M3, 32, 1024",Dell,743.18,"{""CPU"": ""M3"", ""RAM_GB"": 32, ""Storage_GB"": 1024}"


In [13]:
print(f"""
* After Merging *
merged dataframe shape: {merged_df.shape[0]} × {merged_df.shape[1]}
""")


* After Merging *
merged dataframe shape: 50000 × 14



# Saving

In [14]:
merged_df = merged_df.sort_values(by=['CustomerID', 'Timestamp', 'SKU'])

In [15]:
mem_MB = merged_df.memory_usage(deep=True).sum() / 1_048_576  # bytes → MB
print(f"Estimated in-memory size: {mem_MB:.2f} MB")

Estimated in-memory size: 35.68 MB


In [16]:
print("Writing merged dataset to CSV")
output_path = "/kaggle/working/merged_transactions"
merged_df.to_csv(output_path + ".csv", index=False)
print("Merged dataset saved successfully")

Writing merged dataset to CSV
Merged dataset saved successfully


# Verification

In [17]:
import unittest

class TestETLIntegrity(unittest.TestCase):

    def test_customerid_integrity(self):
        missing_customers = transactions_log.loc[~transactions_log['CustomerID'].isin(customers['CustomerID'])]
        self.assertTrue(missing_customers.empty, f"❌ Found {len(missing_customers)} transactions with missing CustomerIDs.")

    def test_sku_integrity(self):
        missing_skus = transactions_log.loc[~transactions_log['SKU'].isin(products_catalog['SKU'])]
        self.assertTrue(missing_skus.empty, f"❌ Found {len(missing_skus)} transactions with missing SKUs.")

# Run the tests
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_customerid_integrity (__main__.TestETLIntegrity.test_customerid_integrity) ... ok
test_sku_integrity (__main__.TestETLIntegrity.test_sku_integrity) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.012s

OK
